# Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matricesRalas import *

### Datos del Grafo

In [7]:
def fillMatrixEJ3():
    mat = MatrizRala(11,11)
    # CITAS A-0
    mat[0,2] = 1
    mat[0,3] = 1
    mat[0,4] = 1

    # CITAS B-1
    mat[1,0] = 1

    # CITAS C-2

    # CITAS D-3

    # CITAS E-4
    mat[4,10] = 1

    # CITAS F-5
    mat[5,0] = 1
    mat[5,6] = 1

    # CITAS G-6
    mat[6,7] = 1
    mat[6,8] = 1

    # CITAS H-7
    mat[7,8] = 1

    # CITAS I-8
    mat[8,5] = 1

    # CITAS J-9
    mat[9,8] = 1
 

    # CITAS K-10

    return mat

Crear las matrices $\mathbf{W}$ y $\mathbf{D}$ para la  utilizando la clas MatrizRala

In [8]:
W = fillMatrixEJ3()
D = W.getD()

Para expresar la ecuación como un sistema lineal $Ax = b$, primero necesitamos reorganizarla. Dado que $p^*$ es el vector de probabilidades que queremos encontrar, podemos escribir la ecuación como:

$p^* = \frac{1 - d}{N} \mathbf{1} + d \mathbf{W}\mathbf{D}p^*$

donde:
- $\mathbf{1}$ es un vector de unos de tamaño $Nx1$.
- $\mathbf{W}$ es la matriz que indica que paper cito al otro.
- $\mathbf{D}$ es la matriz diagonal que contiene la probabilidad dada que un paper sea citado en algun paper dado.
- $d$ probabilidad aleatoria de continuar leyendo alguno de los trabajos citados 
- $N$ es la cantidad de papers

Ahora, podemos reorganizar esta ecuación para obtener:

$p^* - d \mathbf{W}\mathbf{D}p^* = \frac{1 - d}{N} \mathbf{1}$

$p^* \left(\mathbf{I} - d \mathbf{W}\mathbf{D}\right) = \frac{1 - d}{N} \mathbf{1}$


Ahora, podemos expresar esto como un sistema lineal $Ax = b$, donde:
- $A = I - d\mathbf{W}\mathbf{D}$, siendo $I$ la matriz identidad.
- $x = p^*$.
- $b = \frac{1 - d}{N} \mathbf{1}$.

Esta es la forma en que podemos expresar la ecuación como un sistema lineal. Ahora, podemos utilizar métodos numéricos para resolver este sistema y encontrar el vector de probabilidades $p^*$.


Si el sistema converge, encontraremos un vector de probabilidades p∗ con matrices ralas. Expresar la ecuacion como un sistema lineal (Ax = b) y resolver para p∗ utilizando d = 0, 85

In [9]:
d = 0.85
N = W.shape[0]
I = MatrizRala.One(N)

A = I -  d * W @ D #(d*(np.dot(W,D)))
    
vector_1 = MatrizRala.getVectorOne(N)

b = MatrizRala.getVectorOne(N) * ((1-d)/N)

# Metodo gauss
p_sol = GaussJordan(A,b)
print(p_sol)

# Metodo Inversa
invA = A.inversa()
p_sol = invA @ b

print(p_sol)




TypeError: 'tuple' object does not support item assignment

Comparar el resultado obtenido con el metodo iterativo utilizando una distribucion equiprobable para la probabilidad inicial. Para realizar la comparaci´on, graficar la diferencia absoluta entre $p_{t}$ y $p^*$ para valores de t hasta ver que converge.

### Metodo Iterativo

$p^* = \frac{1 - d}{N} \mathbf{1} + d \mathbf{W}\mathbf{D}p^*$

In [ ]:
#calculo utilizando metodo iterativo 

# Vector de probabilidades inicial con distribución equiprobable
p = MatrizRala.getVectorOne(W.shape[0]) * (1/W.shape[0])
    
unos = MatrizRala.getVectorOne(W.shape[0])
    
# Método iterativo
tolerancia = 0.000001
max_iteraciones = 100
diferencias = []
diferencias2 = []

# Calculos Previos
dWD = d * W @ D
parte_div = (1 - d) / N

tiempo = 0

for t in range(max_iteraciones):
    p_previo = p.__copy__()
    
    dWDP_t = dWD @ p_previo
    p = parte_div * unos + dWDP_t
    
    # Calcular la diferencia absoluta entre p y p*
    dif = MatrizRala.diffVectors(p,p_previo)
    dif2 = MatrizRala.diffVectors(p,p_sol)
    
    diferencias.append(dif)
    diferencias2.append(dif2)

    tiempo += 1
    
    if dif < tolerancia:
        print(f"Convergencia alcanzada en la iteración {t + 1}.")
        print("Resultado")
        print(p)
        break



    

AttributeError: 'Nodo' object has no attribute 'raiz'

## Graficos

In [ ]:
def build_graph(rangex:int, y_axis:list):
    """Toma un valor y una lista y genera un grafico

    Args:
        rangex (int): Canitdad de iteraciones
        y_axis (list): Un vector que lleva las diferencias entre p y pt
    """
    # Graficar la diferencia absoluta vs. t
    plt.figure()
    plt.plot(range(rangex), y_axis, label='Diferencia Absoluta')
    plt.xlabel('Iteraciones (t)')
    plt.ylabel('Diferencia Absoluta')
    plt.title('Convergencia del Método Iterativo')
    plt.legend()
    plt.grid(True)
    plt.show() 
    


In [ ]:
build_graph(tiempo,diferencias)

In [ ]:
build_graph(tiempo,diferencias2)

# Tests Rapidos

In [4]:
vector = MatrizRala.getVectorOne(10)
for i in range(10):
    vector[i,0] = i
# print(vector)
vectorNumpy = vector.toNumpy()
# ordenar de mayor a menor
argSort = np.argsort(vectorNumpy, axis=0)[::-1]# ordena de mayor a menor
vectorNumpy = np.sort(vectorNumpy, axis=0)[::-1]

# print(argSort)

vectorNumpy
vectorRala = MatrizRala.fromNumpy(vectorNumpy)
print(vectorRala)

TypeError: 'tuple' object does not support item assignment

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matricesRalas import *
import itertools
papers = 'papers/papers.csv'
citas = 'papers/citas.csv'
citas2 = 'papers/citas2.csv'

W = MatrizRala.getW(papers,citas2)
print(len(W.filas.keys()))
# fila_i = W.filas.values().__iter__().__next__()
# print(fila_i)
# filas = W.filas
# for i in filas:
#     print(i)
#     print(type(i))
#     print(W.filas[436404])
#     break

W[3,2]




NameError: name 'MatrizRala' is not defined

## W D y Paralelismo

In [1]:
from multiprocess import Manager,Pool,cpu_count
import sys
sys.setrecursionlimit(22000)  
def dividir_diccionario(dic, n):
    """Divide el diccionario en n partes aproximadamente iguales."""
    print(f"CANT KEYS: {len(dic.keys())}")
    it = iter(dic)
    tamaño = len(dic)
    tamaño_parte = tamaño // n + (tamaño % n > 0)  # Determina el tamaño de cada parte, ajustando para que sean aproximadamente iguales
    partes = []
    for i in range(n):
        # Toma una porción del tamaño calculado del iterador
        parte = {k: dic[k] for k in itertools.islice(it, tamaño_parte)}
        partes.append(parte)
    return partes

def procesar_fragmento_diccionario(shared_dict,fragmento):
    # Supongamos que queremos contar cuántos valores cumplen cierta condición
    contador = 0
    for key in fragmento.keys():
        contador = 0
        fila = fragmento[key]
        nodo = fila.raiz
        while nodo:
            if(nodo.valor[1] == 1):
                contador += 1
            nodo = nodo.siguiente
        # Agregar contador a la matriz: mat_D[key,key] = contador
        if(contador != 0):
            shared_dict[key] = 1/contador
    
    #     if cumple_condicion(value):  # Define tu propia condición
    #         contador += 1
    # return contador

def D_with_readCsvMultithread(W):
    n_cpu = cpu_count()
    m,n = W.shape
    
    chunks: list = dividir_diccionario(W.filas,n_cpu)
    D = MatrizRala(m,n)
    for chunk in chunks:
        print(len(chunk))
    with Manager() as manager:
        print("LLEGUE")
        shared_dict = manager.dict()
        with Pool(n_cpu) as pool:
            pool.starmap(procesar_fragmento_diccionario,[(shared_dict,chunks[i]) for i in range(len(chunks))])
        
        for key, value in shared_dict.items():
            D[key,key] = value
    return D

D = D_with_readCsvMultithread(W)



NameError: name 'W' is not defined

In [ ]:
i = 629634
fila_i = D.filas[i]
print(fila_i)
len(D.filas.keys())

In [ ]:




top_10 = np.zeros(10)
top_10_index = np.zeros(10, dtype=float)

for i in range(W.shape[0]):
    v = D[i,i]
    if v > top_10[0]:  # Si v es mayor que el más pequeño en top_10
        top_10[0] = v
        top_10_index[0] = i
        sort_indices = np.argsort(top_10)  # Ordena y obtiene índices
        top_10 = top_10[sort_indices]  # Reordena top_10
        top_10_index = top_10_index[sort_indices]  # Reordena los índices de acuerdo a top_10

print(top_10,"\n",top_10_index)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] 
 [140. 134.  89.  85.  75.  57.  34.  25.  20.  11.]


In [ ]:
fila_max = W.filas[top_10_index[-1]]
contador = 0
nodo = fila_max.raiz
while nodo:
    contador += 1
    nodo = nodo.siguiente
print(contador)

In [12]:
i = 629634 + 1
# fila_i = D.filas[i]
# print(fila_i)
# len(D.filas.keys())
i in W.filas.keys()

False

In [10]:
from matricesRalas import *

A = MatrizRala(3,3)
A[0,0] = 1
A[0,0]

1